<a href="https://www.kaggle.com/code/mustafagull/simple-voice-classification?scriptVersionId=166485829" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Veri Yükleme ve Önişleme

In [1]:
from scipy.io import wavfile
import tensorflow as tf
import numpy as np # linear algebra
import os
from tensorflow.keras import layers
import keras
from sklearn.model_selection import train_test_split

X=[]
Y=[]
samplerate, _ = wavfile.read("/kaggle/input/turkish-speech-command-dataset/database/ac/ac_AEMH_OFXZVJN.wav")
print(min(_))

i=0

basepath = '/kaggle/input/turkish-speech-command-dataset/database/'
for entry in os.listdir(basepath):
    
    if os.path.isdir(os.path.join(basepath, entry)):
        
        i+=1
        for entry2 in os.listdir(os.path.join(basepath, entry)):
            
            _, data = wavfile.read(os.path.join(basepath, entry, entry2))
            
            X.append(data/32767)
            Y.append(i)
        
            
            

X = np.vstack(X) 
Y = np.vstack(Y)    
Y=Y.reshape(-1)
X=np.expand_dims(X,axis=-1)

print(X.shape)        
print(Y.shape)        
        

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


-26637
(26484, 16000, 1)
(26484,)


## Eğitim

In [2]:

def network():
    
    inp = layers.Input(shape=(16000,1))
    
    
    #x = layers.MaxPooling1D(pool_size=4, strides=4,padding="valid")(inp)

    x = layers.Conv1D(filters=128, kernel_size=64, strides=4, activation="relu")(inp)
    
    x = layers.MaxPooling1D(pool_size=4, strides=2,padding="same")(x)

    
    x = layers.Dropout(0.3)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    
    x = layers.Conv1D(filters=256, kernel_size=32, strides=4, activation="relu")(x)
    
    x = layers.Dropout(0.3)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    
    x = layers.GlobalAveragePooling1D()(x)
    
    #x = layers.Flatten()(x)
    
    out=keras.layers.Dense(15)(x)
    
    model = keras.Model(inputs=inp, outputs=out)
    return model


model=network()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print(model.summary())
history=model.fit(x=X_train, y=y_train, validation_data=(X_test,y_test) , batch_size=64 ,epochs=15 ,validation_split=0.3)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16000, 1)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 3985, 128)         8320      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1993, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 1993, 128)         0         
_________________________________________________________________
layer_normalization (LayerNo (None, 1993, 128)         256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 491, 256)          1048832   
_________________________________________________________________
dropout_1 (Dropout)          (None, 491, 256)          0     